<a href="https://colab.research.google.com/github/KoreTane/MKTBank/blob/main/MKTBank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install pyspark


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=fa760509d38235a407ee1c4e29ade19d4a0cb58c120cbec5f41fd73efd50f54e
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


# Exploratory Analysis


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.sql.functions import col, count, when
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("BankMarketing").getOrCreate()
df = spark.read.csv("bank.csv", header=True, inferSchema=True)
# Criar um ID único para cada cliente
df = df.withColumn("client_id", monotonically_increasing_id())

colunas = ['client_id'] + [col for col in df.columns if col != 'client_id']
df = df.select(colunas)

df.show(50)

df.printSchema()

df.summary().show()

+---------+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|client_id|age|         job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---------+---+------------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|        0| 59|      admin.| married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
|        1| 56|      admin.| married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
|        2| 41|  technician| married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
|        3| 55|    services| married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| un

In [7]:
# Criar um ID único para cada cliente
df = df.withColumn("client_id", monotonically_increasing_id())

# Mostrar as primeiras linhas do DataFrame com o novo ID
df.show(5)


+---------+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|client_id|age|       job|marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---------+---+----------+-------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|        0| 59|    admin.|married|secondary|     no|   2343|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|    yes|
|        1| 56|    admin.|married|secondary|     no|     45|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|    yes|
|        2| 41|technician|married|secondary|     no|   1270|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|    yes|
|        3| 55|  services|married|secondary|     no|   2476|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|    yes|
|     

In [8]:
# ContaR o número total de registros
total_count = df.count()

# Contar o número de registros distintos
distinct_count = df.distinct().count()

# Verificar se há duplicatas
if total_count > distinct_count:
    print("Existem dados duplicados no DataFrame.")
else:
    print("Não existem dados duplicados no DataFrame.")

Não existem dados duplicados no DataFrame.


In [12]:
# Análises
# 1. Contar quantas IDs têm mais de 80 anos
age_above_80 = df.filter(df.age > 80).count()

# 2. Contar IDs agrupados por job
job_count = df.groupBy("job").count().sort(F.desc("count"))

# 3. Contar quantas IDs estão com balance negativo e positivo
balance_count = df.groupBy(F.when(df.balance < 0, "Negativo").otherwise("Positivo").alias("Balance")).count()

# 4. Contar quantos IDs possuem housing e quantos não
housing_count = df.groupBy("housing").count()

# 5. Contar quantos IDs possuem loan e quantos não
loan_count = df.groupBy("loan").count()

# 6. Contagem do número de IDs agrupada por contact
contact_count = df.groupBy("contact").count()

# Exibindo os resultados em uma matriz

print("Análise de Clientes:")
print(f"\n1. IDs com mais de 80 anos: {age_above_80}")

print("\n2. Contagem de IDs agrupados por Job:")
job_count.show()

print("\n3. Contagem de IDs com Balance Negativo e Positivo:")
balance_count.show()

print("\n4. Contagem de IDs com Housing:")
housing_count.show()

print("\n5. Contagem de IDs com Loan:")
loan_count.show()

print("\n6. Contagem de IDs agrupados por Contact:")
contact_count.show()

Análise de Clientes:

1. IDs com mais de 80 anos: 55

2. Contagem de IDs agrupados por Job:
+-------------+-----+
|          job|count|
+-------------+-----+
|   management| 2566|
|  blue-collar| 1944|
|   technician| 1823|
|       admin.| 1334|
|     services|  923|
|      retired|  778|
|self-employed|  405|
|      student|  360|
|   unemployed|  357|
| entrepreneur|  328|
|    housemaid|  274|
|      unknown|   70|
+-------------+-----+


3. Contagem de IDs com Balance Negativo e Positivo:
+--------+-----+
| Balance|count|
+--------+-----+
|Negativo|  688|
|Positivo|10474|
+--------+-----+


4. Contagem de IDs com Housing:
+-------+-----+
|housing|count|
+-------+-----+
|     no| 5881|
|    yes| 5281|
+-------+-----+


5. Contagem de IDs com Loan:
+----+-----+
|loan|count|
+----+-----+
|  no| 9702|
| yes| 1460|
+----+-----+


6. Contagem de IDs agrupados por Contact:
+---------+-----+
|  contact|count|
+---------+-----+
|  unknown| 2346|
| cellular| 8042|
|telephone|  774|
+--------